# Map CF compliant nc file to rest-like data tree

In [ ]:
import xarray as xr
import numpy as np
import json
from datetime import datetime
from urllib.parse import urljoin

import os
import io

In [ ]:
product_group = 'meteosim'
product_class = 'moloch'
instance_uid = 'AAABBBBCCC3090'
lonlat = 'lon:4.5:512:0.0226_lat:36:512:0.0226'
fname = './data/moloch_2018050103-lonlat.cf'

url_root = 'https://rest.tdm-project.it'

In [ ]:
import gdal, osr
gdal.UseExceptions()

# FIXME check that velocity fields are ok

class MemMasterBuilder(object):
    def __init__(self):
        self.driver = gdal.GetDriverByName("MEM")

    def get_geotransform(self, lons, lats):
        # xstart, ystart is at upper left
        return (lons[0], (lons[-1] - lons[0])/len(lons), 0,
                lats[-1], 0, -(lats[-1] - lats[0])/len(lats))
    
    def setup_raster(self, data, lons, lats):
        rows, cols = data[0].shape
        raster = self.driver.Create("", cols, rows, len(data),  gdal.GDT_Float32)
        raster.SetGeoTransform(self.get_geotransform(lons, lats))
        raster_srs= osr.SpatialReference()
        raster_srs.ImportFromEPSG(4326)
        raster.SetProjection(raster_srs.ExportToWkt())
        return raster

    def build(self, data, lons, lats, metadata):
        if not isinstance(data, list):
            data = [data]
        raster = self.setup_raster(data, lons, lats)
        for i, d in enumerate(data):
            band = raster.GetRasterBand(1 + i)
            band.WriteArray(d)
            band.FlushCache()
        return raster

In [ ]:
product_root = os.path.join('tdm/odata/product', product_group, 
                            product_class, instance_uid)

def path_to_here(timestamp, lonlat):
    path = '/'.join([product_root, timestamp, lonlat])
    return path

def create_res_description(name, desc, url, dt, lonlat, fname):
    stat = os.stat(fname)
    return {"name": name,
            "description": desc,
            "url": url,
            "format": "TIFF",
            "mimetype": "image/tiff",
            "created": stat.st_ctime,
            "last_modified": stat.st_mtime,
            "size": stat.st_size,
            }

In [ ]:
dataset = xr.open_dataset(fname)

In [ ]:
levs = dataset.coords['lev']
times = dataset.coords['time']

In [ ]:
ns = 1e-9 # nanosecs in a sec
FMT = '%Y-%m-%d_%H:%M:%S'
lons = dataset.coords['lon'].values
lats = dataset.coords['lat'].values

out_driver = gdal.GetDriverByName("GTiff")
builder = MemMasterBuilder()
resources = []
for t in times:
    ts = datetime.utcfromtimestamp(t.values.astype(int) * ns).strftime(FMT)
    sel = {'time': t, 'lev': levs[0]}
    ds = dataset.sel(sel)
    path = path_to_here(ts, lonlat)
    os.makedirs(path, exist_ok=True)
    for field_name, desc, fdata in [('hcc', "Total cloud coverage", 
                                     [ds['hcc'].values]), 
                                    ('uv10', "Wind velocity at 10m", 
                                     [ds['10u'][0].values, ds['10v'][0].values])]:
        raster = builder.build(fdata, lons, lats, 
                               {"TIFFTAG_DATETIME": ts})
        out_path = os.path.join(path, "%s.tif" % field_name)
        out_driver.CreateCopy(out_path, raster)
        url = urljoin(url_root, out_path)
        resources.append(create_res_description(field_name, desc, url, ts, lonlat, out_path))
        print('created %s' % out_path)

desc = {"result": {"resources": resources}}
desc_path = os.path.join(product_root, "description.json")
with io.open(desc_path, "wt") as f:
    f.write(json.dumps(desc, indent=4, sort_keys=False))